# Access the MentalRiskEs data and interact with the server

This notebook has been developed by the [SINAI](https://sinai.ujaen.es/) research group for its usage in the [MentalRiskES](https://sites.google.com/view/mentalriskes2024/) evaluation campaign at IberLEF 2024.

**NOTE 1**: Please visit the [MentalRiskES competition website](https://sites.google.com/view/mentalriskes2024/evaluation) to read the instructions about how to download the data and interact with the server to send the predictions of your system.

**NOTE 2**: Along the code, please replace "URL" by the URL server and "TOKEN" by your personal token.

Remember this is a support to help you to develop your own system of communication with our server. We recommend you to download it as a Python script instead of working directly on colab and adapt the code to your needs.

# Install CodeCarbon package
Read the [documentation](https://mlco2.github.io/codecarbon/) about the library if necessary. Remember that we provide a [CodeCarbon notebook](https://colab.research.google.com/drive/1boavnGOir0urui8qktbZaOmOV2pS5cn6?usp=sharing) with the example in its specific use in our competition.


In [ ]:
!pip install codecarbon

# Import libraries

In [ ]:
import requests, zipfile, io
from requests.adapters import HTTPAdapter, Retry
from typing import List, Dict
import random
import json
import os
import pandas as pd
from codecarbon import EmissionsTracker

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

from tqdm import tqdm
import random
from collections import defaultdict

In [ ]:
!pip install emoji

In [ ]:
import re
import emoji

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self, input_size, h_size, output_dim, dropout=0):
        super().__init__()
        self.input_size = input_size
        self.h_size = h_size
        self.output_dim = output_dim

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=h_size, num_layers=1, batch_first=False,
                           dropout=dropout, bidirectional=False)
        self.classifier = nn.Linear(h_size, output_dim)

    def forward(self, seq_data, seq_lens, state=None):
        # seq_data : (S, N, input_size)
        # seq_lens: (N,) -> numbers between 0 and S-1 -> position of last actual sample before padding
        if state is None:
            state = (
                torch.zeros(1, seq_data.shape[1], self.h_size, device=seq_data.device),
                torch.zeros(1, seq_data.shape[1], self.h_size, device=seq_data.device)
            )

        out_states, _ = self.lstm(seq_data, state) # S, N, H
        pred_states = torch.take_along_dim(out_states, seq_lens, dim=0).squeeze() # remove seq dimension
        out = self.classifier(pred_states)
        return out

    def predict_all_timesteps(self, seq_data, seq_lens, state=None):
        if state is None:
            state = (
                torch.zeros(1, seq_data.shape[1], self.h_size, device=seq_data.device),
                torch.zeros(1, seq_data.shape[1], self.h_size, device=seq_data.device)
            )

        out_states, _ = self.lstm(seq_data, state) # [S, N, H]
        logits_all = self.classifier(out_states) # [S, N, n_classes]
        pred_all = torch.argmax(logits_all, dim=2) # [S, N]
        ts_predictions = []
        for i in range(pred_all.shape[1]): # batch_dim
            ts_predictions.append(pred_all[:seq_lens[0, i].item(), i].squeeze().cpu().numpy())

        return ts_predictions

    def predict_one_step(self, data, states=None):
        # data: [N, E] -> [1,N,E] # there is only 1 item in each sequence
        # state: None or tuple([N, H], [N, H])
        ## keep states as 1, 1, H in predictor -> easier to concat that way

        if states == None:
            # init states with zeros
            states = (
                torch.zeros(1, data.shape[1], self.h_size, device=data.device, dtype=data.dtype),
                torch.zeros(1, data.shape[1], self.h_size, device=data.device, dtype=data.dtype)
            )

        _, (hn, cn) = self.lstm(data, states) # ([1, N, H], [1, N, H])
        out = self.classifier(hn.squeeze()) # [N, C]
        predictions = torch.argmax(out, axis=-1).cpu().numpy() # [N}]

        return predictions, (hn.permute(1,0,2), cn.permute(1,0,2))

    def predict_one_step_multilabel(self, data, states=None):
        # data: [N, E] -> [1,N,E] # there is only 1 item in each sequence
        # state: None or tuple([N, H], [N, H])

        if states == None:
            # init states with zeros
            states = (
                torch.zeros(1, data.shape[1], self.h_size, device=data.device, dtype=data.dtype),
                torch.zeros(1, data.shape[1], self.h_size, device=data.device, dtype=data.dtype)
            )

        _, (hn, cn) = self.lstm(data, states) # ([1, N, H], [1, N, H])
        out = self.classifier(hn.squeeze()) # [N, C]
        predictions = (out > 0).cpu().numpy() # [N]

        return predictions, (hn.permute(1,0,2), cn.permute(1,0,2))


In [ ]:
textual_emoticons_to_spanish = {
        ":)": "cara sonriente",
        ":(": "cara triste",
        ";)": "guiño",
        ":D": "cara riendo con los ojos abiertos",
        "XD": "cara riendo con los ojos cerrados",
        "xD": "cara riendo con los ojos cerrados",
        ":P": "cara sacando la lengua",
        "<3": "corazón",
        ":'(": "cara llorando",
        ":-)": "cara sonriente",
        ":-(": "cara triste",
        ";-)": "guiño",
        ":-D": "cara riendo con los ojos abiertos",
        ":-P": "cara sacando la lengua",
        "(heart)": "corazón",
        ":o": "cara sorprendida",
        ":-o": "cara sorprendida",
        ":/": "cara de duda"
    }

def preprocess(text):
    # print(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'(ja)+', 'jaja', text, flags=re.IGNORECASE)
    text = re.sub(r'(js)+', 'jsjs', text, flags=re.IGNORECASE)
    text = text.replace(" @","o")
    for x,y in textual_emoticons_to_spanish.items():
        text = text.replace(x,y)
    text = emoji.demojize(text,language='es')
    spanishVowels = 'aeiouáéíóú'
    uppercaseVowels =spanishVowels.upper()
    for vow in spanishVowels + uppercaseVowels:
        pattern = re.compile(f"{vow}{vow}{vow}+")
        text = pattern.sub(f'{vow}',text)
    return text

In [ ]:
def get_cls_embeddings(all_messages, model, tokenizer, device, m_length=96):
    model.to(device)
    model.eval()
    embeddings = []
    with torch.no_grad():
        for subject_messages in tqdm(all_messages):
            input = tokenizer(subject_messages, padding=True, truncation=True, max_length=m_length, return_tensors='pt')
            output = model(**input.to(device))
            embeddings.append(output.last_hidden_state[:, 0, :].cpu().numpy())
    return embeddings

def get_cls_embeddings_2(messages, model, tokenizer, device, m_length=96):
    model.to(device)
    model.eval()
    embeddings = []
    n_messages = len(messages)
    with torch.no_grad():
        for i in tqdm(range(0, n_messages, 128)):
            msg_batch = messages[i: min(i+128, n_messages)]
            input = tokenizer(msg_batch, padding=True, truncation=True, max_length=m_length, return_tensors='pt')
            output = model(**input.to(device))
            embeddings.append(output.last_hidden_state[:, 0, :].cpu().numpy())

    embeddings = np.concatenate(embeddings, axis=0)
    return embeddings

In [ ]:
from functools import partial

def transformer_embedding(encoder, tokenizer, messages, device, max_length=128):
    encoder.eval()
    # encoder.to(device)
    with torch.no_grad():
        input = tokenizer(messages, padding=True, truncation=True, max_length=128, return_tensors='pt')
        output = encoder(**input.to(device))
        cls_embeddings = output.last_hidden_state[:, 0, :]
    return cls_embeddings

model_name = 'pysentimiento/robertuito-sentiment-analysis'
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.to(device)
encoder.eval()
sentiment_emb_func = partial(transformer_embedding, encoder=encoder, tokenizer=tokenizer, device=device)

model_name2 = 'ignacio-ave/beto-sentiment-analysis-spanish'
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
encoder2 = AutoModel.from_pretrained(model_name2)
encoder2.to(device)
encoder2.eval()
context_emb_func = partial(transformer_embedding, encoder=encoder2, tokenizer=tokenizer2, device=device)


In [ ]:
task1_rnn_1 = LSTMClassifier(768, 96, 3)
task1_rnn_1.load_state_dict(torch.load('/path/to/net_params_96_96.pt')) # 0.8 best macro_f1
task1_rnn_1.eval()
task1_rnn_1.to(device)

task1_rnn_2 = LSTMClassifier(768, 128, 3)
task1_rnn_2.load_state_dict(torch.load('/path/to/net_params_128_64.pt')) # 0.797
task1_rnn_2.eval()
task1_rnn_2.to(device)

task1_rnn_preprocess = LSTMClassifier(768, 128, 3)
task1_rnn_preprocess.load_state_dict(torch.load('/path/to/net_params_preprocess_128_32.pt')) #0.7979
task1_rnn_preprocess.eval()
task1_rnn_preprocess.to(device)
None

In [ ]:
task2_rnn_1 = LSTMClassifier(768, 128, 7)
task2_rnn_1.load_state_dict(torch.load('/path/to/best_avg.pth', map_location='cpu'))
task2_rnn_1.to(device)

task2_rnn_2 = [LSTMClassifier(768, 128, 7) for i in range(6)]

for i in range(6):
    task2_rnn_2[i].load_state_dict(torch.load(f'/path/to/best_class_{i}.pth', map_location='cpu'))
    task2_rnn_2[i].to(device)


In [ ]:
LABEL_MAP = {'none':0, 'anxiety':1, 'depression':2}
REVERSE_LABEL_MAP = {0:'none', 1:'anxiety', 2:'depression'}

def process_round_task1(data, embedding_func, rnn, device, state=None):
    # data = [{'nick': 'subject10', 'round': 1, 'id_message': 123, 'message': "...", 'date': "YYYY-MM-DD HH:MM:SS"}]
    subjects = [x['nick'] for x in data]
    messages = [x['message'] for x in data]

    if state is None: # init state here
        state = {
            subject: {
                'hidden_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32), # put on device only when needed
                'cell_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32),
                'task2_response': None,
            } for subject, message in zip(subjects, messages)
        }

    embeddings = embedding_func(messages=messages, device=device).unsqueeze(0) # returns the torch tensor on cuda
    # embeddings.shape = [N, E]

    h_states = [state[subject]['hidden_state'] for subject in subjects] # [N, 1, 1, H]
    h_states = torch.cat(h_states, dim=1).to(device) # [1, N, H] cuda
    c_states = [state[subject]['cell_state'] for subject in subjects] # [N, 1, 1, H]
    c_states = torch.cat(c_states, dim=1).to(device) # [1, N, H] cuda
    with torch.no_grad():
        predictions, (h_new, c_new) = rnn.predict_one_step(embeddings, (h_states, c_states))

    response = {}
    for subject, prediction, h_state, c_state in zip(subjects, predictions, h_new, c_new):
        state[subject]['hidden_state'] = h_state.cpu().reshape(1,1,-1) # state updated in place
        state[subject]['cell_state'] = c_state.cpu().reshape(1,1,-1)
        response[subject] = REVERSE_LABEL_MAP[prediction]

    return response, state


In [ ]:

CONTEXTS = ['addiction','emergency','family','work','social','other','none']
CONTEXT_MAP = {c:i for i,c in enumerate(CONTEXTS)}
REVERSE_CONTEXT_MAP = {CONTEXT_MAP[c]:c for c in CONTEXT_MAP.keys()}

def process_round_task2(data, embedding_func, rnn, device, state=None):
    subjects = [x['nick'] for x in data]
    messages = [x['message'] for x in data]
    embeddings = embedding_func(messages=messages, device=device).unsqueeze(0)

    if state is None: # init state here
        state = {
            subject: {
                'hidden_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32), # put on device only when needed
                'cell_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32),
                'task2_response': None,
            } for subject, message in zip(subjects, messages)
        }
    h_states = [state[subject]['hidden_state'] for subject in subjects] # [N, 1, 1, H]
    h_states = torch.cat(h_states, dim=1).to(device) # [1, N, H]; cuda
    c_states = [state[subject]['cell_state'] for subject in subjects] # [N, 1, 1, H]
    c_states = torch.cat(c_states, dim=1).to(device) # [1, N, H]l cuda

    with torch.no_grad():
        predictions, (h_new, c_new) = rnn.predict_one_step_multilabel(embeddings, (h_states, c_states))

    print(predictions.shape)
    response = {}
    for subject, prediction, h_state, c_state in zip(subjects, predictions, h_new, c_new):
        state[subject]['hidden_state'] = h_state.cpu().reshape(1,1,-1) # state updated in place
        state[subject]['cell_state'] = c_state.cpu().reshape(1,1,-1)

        str_labels = []
        for i in range(6):
            if prediction[i] == 1:
                str_labels.append(REVERSE_CONTEXT_MAP[i])
        if len(str_labels) == 0:
            response[subject] = 'none'
        else:
            response[subject] = '#'.join(str_labels)
        state[subject]['task2_response'] = response[subject] # will always resend this one instead of predicting again;;

    return response, state


In [ ]:

CONTEXTS = ['addiction','emergency','family','work','social','other','none']
CONTEXT_MAP = {c:i for i,c in enumerate(CONTEXTS)}
REVERSE_CONTEXT_MAP = {CONTEXT_MAP[c]:c for c in CONTEXT_MAP.keys()}

def process_round_task2_multipleRNNs(data, embedding_func, rnns, device, states=None):
    subjects = [x['nick'] for x in data]
    messages = [x['message'] for x in data]
    embeddings = embedding_func(messages=messages, device=device).unsqueeze(0)

    if states is None: # init state here
        states = [
            {
            subject: {
                'hidden_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32), # put on device only when needed
                'cell_state': torch.zeros(1, 1, rnn.h_size, dtype=torch.float32),
            } for subject, message in zip(subjects, messages)
        }
        for rnn in rnns]

    all_h_states = []
    all_c_states = []
    for state in states:
        h_states = [state[subject]['hidden_state'] for subject in subjects] # [N, 1, 1, H]
        h_states = torch.cat(h_states, dim=1).to(device) # [1, N, H]; cuda
        all_h_states.append(h_states)

        c_states = [state[subject]['cell_state'] for subject in subjects] # [N, 1, 1, H]
        c_states = torch.cat(c_states, dim=1).to(device) # [1, N, H]l cuda
        all_c_states.append(c_states)

    fullPredictions = None
    for i,rnn in enumerate(rnns):
        with torch.no_grad():
            predictions, (h_new, c_new) = rnn.predict_one_step_multilabel(embeddings, (all_h_states[i], all_c_states[i]))

        state = states[i]
        for subject, prediction, h_state, c_state in zip(subjects, predictions, h_new, c_new):
            state[subject]['hidden_state'] = h_state.cpu().reshape(1,1,-1) # state updated in place
            state[subject]['cell_state'] = c_state.cpu().reshape(1,1,-1)

        currentPredictions = predictions[:,i].reshape(embeddings.shape[1],1)

        if fullPredictions is None:
            fullPredictions = currentPredictions
        else:
            fullPredictions = np.concatenate([fullPredictions, currentPredictions],axis=1)


    response = {}
    for subject, prediction in zip(subjects, fullPredictions):
        str_labels = []
        for i in range(6):
            if prediction[i] == 1:
                str_labels.append(REVERSE_CONTEXT_MAP[i])
        if len(str_labels) == 0:
            response[subject] = 'none'
        else:
            response[subject] = '#'.join(str_labels)

    return response, states


In [ ]:
def process_round_task3(data, embedding_func, class_layer, device, state=None):
    # data = [{'nick': 'subject10', 'round': 1, 'id_message': 123, 'message': "...", 'date': "YYYY-MM-DD HH:MM:SS"}]
    subjects = [x['nick'] for x in data]
    messages = [x['message'] for x in data]
    embeddings = embedding_func(messages=messages, device=device) # embeddings.shape = [N, E]

    with torch.no_grad():
        output = class_layer(embeddings)
        predictions = torch.argmax(output, axis=1).cpu().numpy().astype(np.int32)

    response = {
        subject: int(prediction) for subject, prediction in zip(subjects, predictions)
    }
    return response

In [ ]:
task3_linear_1 = nn.Linear(768, 2)
task3_linear_1.load_state_dict(torch.load('/path/to/model_128_2.pt')) # only none vs suicidal -> mby too confident
task3_linear_1.to(device)

task3_linear_2 = nn.Linear(768, 2)
task3_linear_2.load_state_dict(torch.load('/path/to/model_all_32_1.pt')) # high precision
task3_linear_2.to(device)

task3_linear_3 = nn.Linear(768, 2)
task3_linear_3.load_state_dict(torch.load('/path/to/model_all_64_1.pt')) # high recall
task3_linear_3.to(device)


# Endpoints
These URL addresses are necessary for the connection to the server.

**IMPORTANT:** Replace "URL" by the URL server and "TOKEN" by your user token.

In [ ]:
URL = ""
TOKEN = ""

# Download endpoints
# ENDPOINT_DOWNLOAD_TRIAL = URL+"/{TASK}/download_trial/{TOKEN}"
# ENDPOINT_DOWNLOAD_TRAIN = URL+"/{TASK}/download_train/{TOKEN}"


# Trial endpoints
# ENDPOINT_GET_MESSAGES_TRIAL = URL+"/{TASK}/getmessages_trial/{TOKEN}"
# ENDPOINT_SUBMIT_DECISIONS_TRIAL = URL+"/{TASK}/submit_trial/{TOKEN}/{RUN}"

# Test endpoints
ENDPOINT_GET_MESSAGES = URL+"/{TASK}/getmessages/{TOKEN}"
ENDPOINT_SUBMIT_DECISIONS = URL+"/{TASK}/submit/{TOKEN}/{RUN}"

# Client Server
This class simulates communication with our server. The following code established the conection with the server client and simulate the GET and POST requests.

**IMPORTANT NOTE:** Please pay attention to the basic functions and remember that it is only a base for your system.

In [ ]:
class Client_taskX:
    """ Client communicating with the official server.
        Attributes:
            task (str): task in which you wish to participate
            token (str): authentication token
            number_of_runs (int): number of systems. Must be 3 in order to advance to the next round.
            tracker (EmissionsTracker): object to calculate the carbon footprint in prediction

    """
    def __init__(self, task: str, token: str, number_of_runs: int, tracker: EmissionsTracker, task1_funcs=None, task2_funcs=None):
        self.task = task
        self.token = token
        self.number_of_runs = number_of_runs
        self.tracker = tracker
        # Required parameters
        self.relevant_cols = ['duration', 'emissions', 'cpu_energy', 'gpu_energy',
                              'ram_energy','energy_consumed', 'cpu_count', 'gpu_count',
                              'cpu_model', 'gpu_model', 'ram_total_size','country_iso_code']


    def get_messages(self, retries: int, backoff: float, task2_=False) -> Dict:
        """ Allows you to download the test data of the task by rounds.
            Here a GET request is sent to the server to extract the data.
            Args:
              retries (int): number of calls on the server connection
              backoff (float): time between retries
        """
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))
        response = session.get(ENDPOINT_GET_MESSAGES.format(TASK=self.task, TOKEN=self.token)) # ENDPOINT FOR TRIAL

        if response.status_code != 200:
          print("GET - Task {} - Status Code {} - Error: {}".format(self.task, str(response.status_code), str(response.text)))
          return []
        else:
          return json.loads(response.content)


    def get_messages_task2(self, retries: int, backoff: float) -> Dict:
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))
        response = session.get(ENDPOINT_GET_MESSAGES.format(TASK='task2', TOKEN=self.token)) # ENDPOINT FOR TRIAL

        if response.status_code != 200:
          print("GET - Task {} - Status Code {} - Error: {}".format(self.task, str(response.status_code), str(response.text)))
          return []
        else:
          return json.loads(response.content)


    def submit_decission_task3(self, all_predictions: List[Dict], all_emissions: List[Dict], retries: int, backoff: float):
        """ Allows you to submit the decisions of the task by rounds.
            The POST requests are sent to the server to send predictions and carbon emission data
            Args:
              predictions (List[Dict]): predictions for each of the 3 runs
              emissions (Dict): carbon footprint generated in the 3 predictions
              retries (int): number of calls on the server connection
              backoff (float): time between retries
        """
        data = [
            {
                "predictions": predictions,
                "emissions": emissions,
            } for predictions, emissions in zip(all_predictions, all_emissions)
        ]
        json_data = [json.dumps(data_) for data_ in data]

        # Session to POST request
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))

        for run in range(0, self.number_of_runs):
            # For each run, new decisions
            response = session.post(ENDPOINT_SUBMIT_DECISIONS.format(TASK='task3', TOKEN=self.token, RUN=run),
                                    json=[json_data[run]]) # ENDPOINT FOR TRIAL

            if response.status_code != 200:
                print("POST - Task {} - Status Code {} - Error: {}".format(self.task, str(response.status_code), str(response.text)))
                return
            else:
                print("POST - Task {} - run {} - Message: {}".format(self.task, run, str(response.text)))


    def run_task3(self, retries: int, backoff: float):
        """ Main thread
            Args:
              retries (int): number of calls on the server connection
              backoff (float): time between retries
        """
        # Get messages for taskX
        messages = self.get_messages(retries, backoff)
        # If there are no messages
        if len(messages) == 0:
            print("All rounds processed")
            return

        while len(messages) > 0:
            print("------------------- Processing round {}".format(messages[0]["round"]))
            all_predictions, all_emissions = [], []

            # Calculate emissions for each prediction
            self.tracker.start()
            # preprocessing should also be a part of this...
            processed_messages = [
                {
                    'message': preprocess(x['message']),
                    'nick': x['nick']
                } for x in messages
            ]
            predictions = process_round_task3(data=processed_messages, embedding_func=sentiment_emb_func,
                                             class_layer=task3_linear_1, device=device)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            all_predictions.append(predictions)
            all_emissions.append(measurements)

            self.tracker.start()
            processed_messages = [
                {
                    'message': preprocess(x['message']),
                    'nick': x['nick']
                } for x in messages
            ]
            predictions = process_round_task3(data=processed_messages, embedding_func=sentiment_emb_func,
                                             class_layer=task3_linear_2, device=device)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            all_predictions.append(predictions)
            all_emissions.append(measurements)


            self.tracker.start()
            processed_messages = [
                {
                    'message': preprocess(x['message']),
                    'nick': x['nick']
                } for x in messages
            ]
            predictions = process_round_task3(data=processed_messages, embedding_func=sentiment_emb_func,
                                             class_layer=task3_linear_3, device=device)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            all_predictions.append(predictions)
            all_emissions.append(measurements)
            # print(all_emissions)

            self.submit_decission_task3(all_predictions, all_emissions, retries, backoff)

            # One GET request for each round
            messages = self.get_messages(retries, backoff)

        print("All rounds processed")



    def submit_decissions_12(self, predictions_task1: List[Dict], emissions_task1: List[Dict],
                             predictions_task2: List[Dict], contexts_task2: List[Dict], emissions_task2: List[Dict],
                             retries: int, backoff: float):
        """ Allows you to submit the decisions of the task by rounds.
            The POST requests are sent to the server to send predictions and carbon emission data
            Args:
              predictions (List[Dict]): Current round task1 predictions
              emissions (Dict): carbon footprint generated in the prediction
              retries (int): number of calls on the server connection
              backoff (float): time between retries
        """
        task1_submissions = []
        for i in range(3):
            data = {
                "predictions": predictions_task1[i],
                "emissions": emissions_task1[i]
            }
            task1_submissions.append(json.dumps(data))

        # Session to POST request
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))

        for run in range(0, self.number_of_runs): # 3
            # For each run, new decisions
            response = session.post(
                ENDPOINT_SUBMIT_DECISIONS.format(TASK='task1', TOKEN=self.token, RUN=run),
                json=[task1_submissions[run]]
            )

            if response.status_code != 200:
                print("POST - Task {} - Status Code {} - Error: {}".format(self.task, str(response.status_code), str(response.text)))
                return
            else:
                print("POST - Task {} - run {} - Message: {}".format(self.task, run, str(response.text)))

        task2_submissions = []
        for i in range(3):
            data = {
                "predictions": predictions_task2[i],
                "contexts": contexts_task2[i],
                "emissions": emissions_task2[i]
            }
            task2_submissions.append(json.dumps(data))

        # Session to POST request
        session = requests.Session()
        retries = Retry(
                        total = retries,
                        backoff_factor = backoff,
                        status_forcelist = [500, 502, 503, 504]
                        )
        session.mount('https://', HTTPAdapter(max_retries=retries))

        for run in range(0, self.number_of_runs):
            # For each run, new decisions
            response = session.post(
                ENDPOINT_SUBMIT_DECISIONS.format(TASK='task2', TOKEN=self.token, RUN=run),
                json=[task2_submissions[run]]
            )

            if response.status_code != 200:
                print("POST - Task {} - Status Code {} - Error: {}".format(self.task, str(response.status_code), str(response.text)))
                return
            else:
                print("POST - Task {} - run {} - Message: {}".format(self.task, run, str(response.text)))


    def run_task_12(self, retries: int, backoff: float):
        """ Main thread
            Args:
              retries (int): number of calls on the server connection
              backoff (float): time between retries
        """
        state1 = None
        state2 = None
        state3 = None

        state21 = None
        state22 = None
        state23 = None
        # Get messages for taskX
        messages = self.get_messages(retries, backoff)
        # If there are no messages
        if len(messages) == 0:
            print("All rounds processed")
            return

        while len(messages) > 0:
            print("------------------- Processing round {}".format(messages[0]["round"]))
            # Calculate emissions for each prediction
            predictions_task1 = []
            emissions_task1 = []

            self.tracker.start()
            predictions, state1 = process_round_task1(data=messages, embedding_func=sentiment_emb_func,
                                                         rnn=task1_rnn_1, device=device, state=state1)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task1.append(measurements)
            predictions_task1.append(predictions)

            self.tracker.start()
            predictions, state2 = process_round_task1(data=messages, embedding_func=sentiment_emb_func,
                                                         rnn=task1_rnn_2, device=device, state=state2)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task1.append(measurements)
            predictions_task1.append(predictions)


            self.tracker.start()
            preprocessed_messages = [
                {
                    'nick': x['nick'],
                    'message': preprocess(x['message'])
                } for x in messages
            ]
            predictions, state3 = process_round_task1(data=preprocessed_messages, embedding_func=sentiment_emb_func,
                                                         rnn=task1_rnn_preprocess, device=device, state=state3)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task1.append(measurements)
            predictions_task1.append(predictions)


            messages_2 = self.get_messages_task2(retries, backoff)
            assert messages[0]['round'] == messages_2[0]['round'], f'Round numbers mismatch between tasks 1 and 2 | {messages[0]["round"]} {messages_2[0]["round"]}'
            ### TASK2
            predictions_task2 = [predictions_task1[2], predictions_task1[2], predictions_task1[0]]
            contexts_task2 = []
            emissions_task2 = []

            self.tracker.start()
            preprocessed_messages = [
                {
                    'nick': x['nick'],
                    'message': preprocess(x['message'])
                } for x in messages
            ]
            predictions, state21 = process_round_task2(data=preprocessed_messages, embedding_func=context_emb_func,
                                                         rnn=task2_rnn_1, device=device, state=state21)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task2.append(measurements)
            contexts_task2.append(predictions)

            self.tracker.start()
            preprocessed_messages = [
                {
                    'nick': x['nick'],
                    'message': preprocess(x['message'])
                } for x in messages
            ]
            predictions, state22 = process_round_task2_multipleRNNs(data=preprocessed_messages, embedding_func=context_emb_func,
                                                         rnns=task2_rnn_2, device=device, states=state22)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task2.append(measurements)
            contexts_task2.append(predictions)


            self.tracker.start()
            preprocessed_messages = [
                {
                    'nick': x['nick'],
                    'message': preprocess(x['message'])
                } for x in messages
            ]
            predictions, state23 = process_round_task2(data=preprocessed_messages, embedding_func=context_emb_func,
                                                         rnn=task2_rnn_1, device=device, state=state23)
            emissions = self.tracker.stop()
            df = pd.read_csv("emissions.csv")
            measurements = df.iloc[-1][self.relevant_cols].to_dict()
            emissions_task2.append(measurements)
            contexts_task2.append(predictions)


            # finally, submit the predictions
            self.submit_decissions_12(predictions_task1, emissions_task1,
                                      predictions_task2, contexts_task2, emissions_task2,
                                      retries, backoff)

            # One GET request for each round
            messages = self.get_messages(retries, backoff)

        print("All rounds processed")


# Main

In [ ]:
task = 'task1'

config = {
    "save_to_file": True,
    "log_level": "DEBUG",
    "tracking_mode": "process",
    "output_dir": ".",
}
tracker = EmissionsTracker(**config)

client_taskX = Client_taskX(task, TOKEN, 3, tracker)

In [ ]:
client_taskX.run_task_12(5, 0.1)

In [ ]:
task = 'task3'

config = {
    "save_to_file": True,
    "log_level": "DEBUG",
    "tracking_mode": "process",
    "output_dir": ".",
}
tracker = EmissionsTracker(**config)

client_taskX = Client_taskX(task, TOKEN, 3, tracker)

In [ ]:
client_taskX.run_task3(5, 0.1)